# Random Forest

In [22]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [23]:
df = pd.read_csv("data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [24]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
                        'koi_period','koi_period_err1','koi_period_err2',
                        'koi_time0bk','koi_time0bk_err1','koi_time0bk_err2',
                        'koi_impact','koi_impact_err1','koi_impact_err2',
                        'koi_duration','koi_duration_err1','koi_duration_err2',
                        'koi_depth','koi_depth_err1','koi_depth_err2',
                        'koi_prad','koi_prad_err1','koi_prad_err2',
                        'koi_teq','koi_insol','koi_insol_err1','koi_insol_err2',
                        'koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2',
                        'koi_slogg','koi_slogg_err1','koi_slogg_err2',
                        'koi_srad','koi_srad_err1','koi_srad_err2',
                        'ra','dec','koi_kepmag']]
selected_features.head(5)

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [25]:
target = df["koi_disposition"]
target_names = ["CONFIRMED", "FALSE POSITIVE", "CANDIDATE"]

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42, stratify=target)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [27]:
# Scale data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler

# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


# Train the Model


In [28]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_scaled, encoded_y_train)

C:\Users\fabio\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
print(f"Training Data Score: {rf.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.9944688155636087
Testing Data Score: 0.8895881006864989


In [30]:
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.09781319251063358, 'koi_fpflag_co'),
 (0.09527646550870714, 'koi_fpflag_nt'),
 (0.0733259101799124, 'koi_prad'),
 (0.05923993960558648, 'koi_duration_err1'),
 (0.05857571381419251, 'koi_model_snr'),
 (0.05316412602979106, 'koi_fpflag_ss'),
 (0.04637247834861974, 'koi_fpflag_ec'),
 (0.04464651853138332, 'koi_prad_err1'),
 (0.03743273502197486, 'koi_steff_err1'),
 (0.03505175148935241, 'koi_steff_err2'),
 (0.02401119900993112, 'koi_period_err2'),
 (0.023811529010132264, 'koi_duration'),
 (0.022445378806122403, 'koi_impact'),
 (0.02244253251525938, 'koi_insol_err1'),
 (0.021641655881532523, 'koi_depth'),
 (0.018722895728153204, 'koi_time0bk_err1'),
 (0.017626357260761314, 'koi_period'),
 (0.016879467882094875, 'koi_time0bk'),
 (0.015161518149841747, 'koi_period_err1'),
 (0.01513820404886975, 'koi_depth_err1'),
 (0.013978317563735283, 'koi_duration_err2'),
 (0.013651338830270828, 'koi_srad_err1'),
 (0.013258896676400662, 'koi_teq'),
 (0.012137457868513472, 'koi_impact_err1'),
 (0.01203

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [39]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [250, 300, 350],
              'max_depth': [125, 150, 175]}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [40]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

C:\Users\fabio\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.880, total=   6.4s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.899, total=   6.7s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.0s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.888, total=   6.5s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.884, total=   7.8s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.892, total=   7.8s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.883, total=   7.8s
[CV] max_depth=125, n_estimators=350 .................................
[CV] ..... max_depth=125, n_estimators=350, score=0.887, total=   9.0s
[CV] max_depth=125, n_estimators=350 .................................
[CV] ..... max_depth=125, n_estimators=350, score=0.896, total=   9.3s
[CV] max_depth=125, n_estimators=350 .................................
[CV] ..... max_depth=125, n_estimators=350, score=0.882, total=   9.0s
[CV] max_depth=150, n_estimators=250 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 175, 'n_estimators': 350}
0.8888041197787526


In [33]:
print(f"Training Data Score: {grid.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {grid.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8993135011441648


In [34]:
# Make prediction and save to variable for report.
predictions = grid.predict(X_test_scaled)

In [35]:
# Print Classification Report.
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79       422
           1       0.81      0.84      0.83       450
           2       0.97      1.00      0.99       876

    accuracy                           0.90      1748
   macro avg       0.87      0.86      0.87      1748
weighted avg       0.90      0.90      0.90      1748



# Save the Model

In [38]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'models/random_forest.sav'
joblib.dump(rf, filename)

['models/random_forest.sav']